# Brain maps dashboard 
This tutorial shows how to use a NIAK pipeline to generate a dashboard with different brain maps presented as interactive volumetric viewers. 
  * **More documentation**: in the help of the pipeline, see [code on GitHub](https://github.com/SIMEXP/niak/blob/master/reports/brain_map/niak_brick_report_brain_map.m). 
  * **.m script**: The notebook and associated Matlab script are available [here](https://nbviewer.jupyter.org/github/SIMEXP/niak_tutorials/blob/master/niak_tutorial_dashboard_brain_map.ipynb).
  Pre-requisites**: before running this tutorial, you need to complete the [fMRI preprocessing tutorial]
  * **Time for completion**: this tutorial will take 5-10 minutes to complete. 

In [1]:
clear
path_data = pwd;
cd
build_path niak psom
cd(path_data)

Adding library niak to the search path.

Adding library psom to the search path.



## Preparing files

### Background
The background is a 3D volume, typically a structural scan. Here we are just going to use the brain template that ships with NIAK (MNI ICBM152 2009).

In [2]:
clear
path_data = [pwd filesep];
niak_gb_vars
in.background = [GB_NIAK.path_niak filesep 'template' ...
                 filesep 'mni-models_icbm152-nl-2009-1.0' filesep ...
                 'mni_icbm152_t1_tal_nlin_sym_09a.mnc.gz'];

## Overlays
We will downlowd a few preprocessed fMRI volumes, to have something to use as overlay. 

In [3]:
[status,msg,data_fmri] = niak_wget('cobre_lightweight20_nii');

We are going to select two random subjects to generate two overlays. 

In [4]:
in.overlay{1} = [data_fmri.path filesep 'fmri_40003.nii.gz'];
in.overlay{2} = [data_fmri.path filesep 'fmri_40008.nii.gz'];

Note that the overlay volumes are assumed to be in the same space (coregistered) with the background, but do not need to have identical sampling grid. 

## Options
We will start by specifying labels for each overlay, which will appear as title for each corresponding viewer in the dashboard.

In [5]:
opt.labels{1} = 'volume10_subject_40003';
opt.labels{2} = 'volume5_subject_40008';

Next we need to tell NIAK where to generate the dashboard:

In [6]:
opt.folder_out = [path_data filesep 'dashboard_brain_map'];

Let's specify some range parameters and a colormap for the viewers. These cannot be adjusted interactively in the viewer. The pipeline will need to re-run with different options in order to adjust the overlay parameters.

In [7]:
opt.overlay.limits = [0 1300];
opt.overlay.thresh = 100;
opt.overlay.colormap = 'jet';

 * The first parameter `limits` sets the min and max for the overlay. Any value below 0 will have the lowest color in the map. Any value above 1300 will have the max color. 
 * The second parameter `thresh` sets a minimum on values to be displayed anything below that is transparent. Note that it is also possible to remove a range of values: `[-0.3 0.3]` would mask out all values between $-0.3$ and $0.3$. Finally, `colormap` is the colormap to use for the overlay. Here we selected the classis (but controversial) jet, but any Octave colormap will work.
 * Note that equivalent parameters exist for the background, specifically `opt.background.colormap` and `opt.background.limits`. But for structural scans the default values (gray colormap, and adaptative min max) should work fine. 

Now we select the colors for the text and the viewers. Here the structural scan has a black background and we will use a white text. 

In [8]:
opt.color_background = '#FFFFFF'; % those are hexadecimal red/green/blue codes
opt.color_font = '#000000';

We select the number of viewers per row. Different numbers can be specified for different screen size. There are 12 columns available in the layout. Selecting a size of 3 means that each viewer uses 3 columns, and thus 4 viewers will fit on a single row. Here we go with two viewers side by side. See this [documentation](https://www.w3schools.com/bootstrap/bootstrap_grid_system.asp) to learn more on the bootstrap gird layout. 

In [9]:
opt.class_viewer = 'col-lg-6';

Because the fMRI datasets are 4D (space+time), we need to select which volume to display as an overlay, for each brain viewer. 

In [10]:
opt.ind = [10 5];

if a single number was specified, the same volume would be used for all viewers. Here we will look at volume number 10 for the first subject, and volume number 5 for the second one. 

# Run the pipeline
Now let's run the pipeline:

In [11]:
opt.flag_test = false; 
pipeline = niak_report_brain_map(in,opt); 


Logs will be stored in /sandbox/home/git/niak_tutorials/dashboard_brain_map/logs/
Generating dependencies ...
       Reorganizing inputs/outputs ... 0.01 sec
       Analyzing job inputs/outputs... 0.00 sec
Setting up the to-do list ...
   I found 4 job(s) to do, and 1 job(s) already completed.
Deamon started on 07-Jun-2017 16:54:34
07-Jun-2017 16:54:34 Starting the pipeline manager...
07-Jun-2017 16:54:34 Starting the garbage collector...
07-Jun-2017 16:54:34 Starting worker number 1...
07-Jun-2017 16:54:34 Starting worker number 2...

Pipeline started on 07-Jun-2017 16:54:35
user: , host: 5269e56ff0d2, system: unix
****************************************
07-Jun-2017 16:54:36 background                     submitted  (1 run | 0 fail | 1 done | 3 left)
07-Jun-2017 16:54:36 overlay_volume10_subject_40003 submitted  (2 run | 0 fail | 1 done | 2 left)
07-Jun-2017 16:54:36 overlay_volume5_subject_40008  submitted  (3 run | 0 fail | 1 done | 1 left)
07-Jun-2017 16:54:49 overlay_volume10_su

# Reviewing outputs

You can open the dashboard by opening the file [dashboard_brain_map/index.html](dashboard_brain_map/index.html). To share that report, simply copy over the folder with all the files. An internet access is not required to use the dashboard. We recommend using Firefox or Chrome for a better experience, although on some OS users may experience some difficulties with one or the other (but hopefully not both).